<a href="https://colab.research.google.com/github/SIM-FYP2025Q2/SIM-Travels/blob/main/sample_api_calls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Overview

This Colab Notebook shows how to interact with Agent-to-Agent (A2A) agents and access the toolsets provided by our Model Context Protocol (MCP) server using the `fastmcp` library and direct API (POST) calls.


1.  **MCP Server Interaction:**
    *   A FastMCP Client is initialized to connect to our MCP Server.
    *   It the available tools, resources, and prompts registered with the MCP server.
3.  **Agent Card Retrieval:**
    *   The notebook fetches and prints the "agent card" for different A2A agents (Flight Offers, Hotel Offers, Transfer Offers). The agent card provides information about the agent's capabilities and how to interact with it.
4.  **Sending Messages to A2A Agents:**
    *   The notebook sends messages to the A2A agents via API calls using the JSON RPC 2.0 standard. It also prints the Agent Response (results)

In [2]:
# Install Packages
!pip install fastmcp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.2/260.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.3/239.3 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.4/224.4 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 23.9 MB/s eta 0:00:00
  Attempting uninstall: werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3


In [16]:
# Import Libraries
import json
import pprint
import random
import datetime
from datetime import datetime as dt

import asyncio
import requests
from fastmcp import Client, FastMCP

# API URLs
AGENT_CARD_URL = "/.well-known/agent-card.json"
MCP_SERVER_URL = "https://mcp-fyp-25-s2-33.lester-liam.cc/mcp"
FLIGHT_OFFERS_A2A = "https://a2a-agents-fyp-25-s2-33.lester-liam.cc/a2a/flight_offers_agent"
HOTEL_OFFERS_A2A = "https://a2a-agents-fyp-25-s2-33.lester-liam.cc/a2a/hotel_offers_agent"
TRANSFER_OFFERS_A2A = "https://a2a-agents-fyp-25-s2-33.lester-liam.cc/a2a/transfer_offers_agent"

In [7]:
# --- MCP Server List Tools ---

# HTTP Server
client = Client(MCP_SERVER_URL)
async def main():
    async with client:
        # Basic server interaction
        await client.ping()

        # List available operations
        tools:list = await client.list_tools()
        resources = await client.list_resources()
        prompts = await client.list_prompts()

        print("Available Tools:")
        for t in tools:
            pprint.pp(t.model_dump_json(), compact=True)
        print("Available Resources:", resources)
        print("Available Prompts:", prompts)

await main()

Available Tools:
('{"name":"search_flight_offers","title":null,"description":"Search for flight '
 'offers using the Amadeus API\\n\\nRequired Parameters:\\n    '
 'originLocationCode: city/airport IATA code from which the traveler will '
 'depart, (e.g. BOS for Boston\\n    destinationLocationCode: IATA code of the '
 'destination city/airport (e.g., BKK for Bangkok)\\n    departureDate: '
 'Departure date in ISO 8601 format (YYYY-MM-DD, e.g., 2023-05-02)\\n    '
 'adults: Number of adult travelers (age 12+), must be 1-9\\n\\nOptional '
 'Parameters:\\n    returnDate: Return date in ISO 8601 format (YYYY-MM-DD), '
 'if round-trip is desired\\n    children: Number of child travelers (age '
 '2-11)\\n    infants: Number of infant travelers (age <= 2)\\n    '
 'travelClass: Travel class (ECONOMY, PREMIUM_ECONOMY, BUSINESS, FIRST)\\n    '
 'nonStop: If true, only non-stop flights are returned\\n    currencyCode: ISO '
 '4217 currency code (e.g., EUR for Euro)\\n\\nReturns:\\n    JSON Stri

# Retrieving Agent card

In [15]:
# --- List Agents' Cards ---
print("===== Flight Offers Agent =====")
agent_card = requests.get(f"{FLIGHT_OFFERS_A2A}{AGENT_CARD_URL}")
pprint.pp(json.dumps(agent_card.json(), indent=2))
print('\n')

===== Flight Offers Agent =====
('{\n'
 '  "capabilities": {},\n'
 '  "defaultInputModes": [\n'
 '    "text/plain"\n'
 '  ],\n'
 '  "defaultOutputModes": [\n'
 '    "text/plain"\n'
 '  ],\n'
 '  "description": "AI Assistant that searches for flight offers",\n'
 '  "name": "flight_offers_agent",\n'
 '  "preferredTransport": "JSONRPC",\n'
 '  "protocolVersion": "0.3.0",\n'
 '  "skills": [\n'
 '    {\n'
 '      "description": "I am an Flight Offers Search Assistant that searches '
 'for flight offers.\\nUsing only my tool to search for flight offers.\\nIf I '
 'do not have the required information, I will ask for more clarifying '
 "information.\\n\\nHere are the 'search_flight_offers' "
 'parameters:\\n\\n**Required Parameters**:\\n- originLocationCode: '
 'city/airport IATA code from which the traveler will depart, (e.g. BOS for '
 'Boston\\n- destinationLocationCode: IATA code of the destination '
 'city/airport (e.g., BKK for Bangkok)\\n- departureDate: Departure date in '
 'ISO 8601 

In [11]:
print("===== Hotel Offers Agent =====")
agent_card = requests.get(f"{HOTEL_OFFERS_A2A}{AGENT_CARD_URL}")
pprint.pp(json.dumps(agent_card.json(), indent=2))
print('\n')

===== Hotel Offers Agent =====
('{\n'
 '  "capabilities": {},\n'
 '  "defaultInputModes": [\n'
 '    "text/plain"\n'
 '  ],\n'
 '  "defaultOutputModes": [\n'
 '    "text/plain"\n'
 '  ],\n'
 '  "description": "AI Assistant that searches for hotel offers",\n'
 '  "name": "hotel_offers_agent",\n'
 '  "preferredTransport": "JSONRPC",\n'
 '  "protocolVersion": "0.3.0",\n'
 '  "skills": [\n'
 '    {\n'
 '      "description": "A specialized agent for searching and providing '
 'information about hotel accommodations. \\nI am a hotel offers agent, using '
 'only my tool to search for hotel offers.\\nIf I do not have the required '
 'information, I can ask the user for more information.\\n\\nHere are the '
 "'search_hotel_offers' parameters:\\n\\n**Required Parameters:**\\n    "
 'cityCode: City IATA code (e.g., BOS for Boston)\\n    checkInDate: Check In '
 'Date in ISO  format (YYYY-MM-DD, e.g., 2023-05-02)\\n    checkOutDate: Check '
 'Out Date in ISO 8601 format (YYYY-MM-DD, e.g., 2023-05-

In [12]:
print("===== Transfer Offers Agent ====")
agent_card = requests.get(f"{TRANSFER_OFFERS_A2A}{AGENT_CARD_URL}")
pprint.pp(json.dumps(agent_card.json(), indent=2))
print('\n')

===== Transfer Offers Agent ====
('{\n'
 '  "capabilities": {},\n'
 '  "defaultInputModes": [\n'
 '    "text/plain"\n'
 '  ],\n'
 '  "defaultOutputModes": [\n'
 '    "text/plain"\n'
 '  ],\n'
 '  "description": "AI Assistant that searches for airport transfer offers",\n'
 '  "name": "transfer_offers_agent",\n'
 '  "preferredTransport": "JSONRPC",\n'
 '  "protocolVersion": "0.3.0",\n'
 '  "skills": [\n'
 '    {\n'
 '      "description": "A specialized agent for searching and providing '
 'information about airport transfer offers. \\nI am a airport transfer offers '
 'agent, use only my tool to search for private transfer offers.\\nIf I do not '
 'have the required information, I can ask the user for more '
 "information.\\n\\nHere are the 'search_airport_transfers' "
 'parameters:\\n\\n**Required Parameters:**\\n    startAddress: The starting '
 'address for the transfer\\n    endAddress: The ending address for the '
 'transfer\\n    startDatetime: The start date and time for the trans

# Making API Calls to A2A Agents using JSON RPC 2.0 Standard

In [32]:
# Returns a Random Future Date (ahead 4-10 days)
def random_date() -> str:
  random_date = dt.now() + datetime.timedelta(days=random.randint(4, 10))
  return random_date.strftime("%d %B, %Y")

22 August, 2025


In [33]:
# Send Messages to Each Agent via API Call
messages = [
    [FLIGHT_OFFERS_A2A , f"Flights from KUL to SIN, {random_date()}, one-way, MYR"],
    [HOTEL_OFFERS_A2A, f"Hotel Offers in London, {random_date()}, 1 room, 1 pax, USD"],
    [TRANSFER_OFFERS_A2A, f"Transfer Offers from KLIA Airport to Concorde Hotel Shah Alam on {random_date()} (12pm), 1 pax, MYR"]
]

for msg in messages:
  agent_url = msg[0]
  params = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "message/send",
    "params": {
      "message": {
        "role": "user",
        "parts": [
          {
            "kind": "text",
            "text": msg[1]
          }
        ],
        "messageId": "9229e770-767c-417b-a0b0-f0741243c589"
      },
      "metadata": {}
    }
  }

  try:
      print(f"Sending Message `[{msg[1]}]` to A2A Agent ...")
      # Send the POST request
      response = requests.post(agent_url, data=json.dumps(params), headers={'Content-Type': 'application/json'})

      # Response[200]
      if response.status_code == 200:
          print("[Success] Message Sent!")
          print(json.dumps(response.json(), indent=2))
      else:
          print(f"[Error] Received status code [{response.status_code}]")
          print(response.text)
  except requests.exceptions.RequestException as e:
      print(f"An error occurred: {e}")

Sending Message `[Flights from KUL to SIN, 24 August, 2025, one-way, MYR]` to A2A Agent ...
[Success] Message Sent!
{
  "id": 1,
  "jsonrpc": "2.0",
  "result": {
    "artifacts": [
      {
        "artifactId": "d79f6341-dc06-4255-99ae-a60fd42b3e31",
        "parts": [
          {
            "kind": "text",
            "text": "*   **Origin:** Kuala Lumpur (SZB) to Singapore (XSP)\n*   **Departure Date:** 2025-08-24\n*   **Adults:** 1\n*   **Currency:** MYR\n\nHere are the flight offers:\n\n*   **Malaysia Airlines (MH) 5477** departs Kuala Lumpur (SZB) at 06:35, arrives in Singapore (XSP) at 08:00. Price: 178.00 MYR. Includes 20kg checked baggage. Operated by Firefly (FY).\n*   **Malaysia Airlines (MH) 5481** departs Kuala Lumpur (SZB) at 07:15, arrives in Singapore (XSP) at 08:40. Price: 208.00 MYR. Includes 20kg checked baggage. Operated by Firefly (FY).\n*   **Malaysia Airlines (MH) 5483** departs Kuala Lumpur (SZB) at 09:25, arrives in Singapore (XSP) at 10:50. Price: 208.00 MYR.